In [3]:
!date

Wed Oct  2 14:18:22 EDT 2024


In [2]:
pwd

'/mmfs1/data/aglinska/BC-ABCD-denoise/Code'

In [62]:
import os
import numpy as np
import pandas as pd
import ants
from tqdm import tqdm
from matplotlib import pyplot as plt

In [94]:
def correlate_columns(arr1, arr2):
    """
    Computes the Pearson correlation between corresponding columns of two matrices.
    
    Parameters:
    arr1 (np.ndarray): First matrix of shape (370, 1000)
    arr2 (np.ndarray): Second matrix of shape (370, 1000)
    
    Returns:
    np.ndarray: 1D array of correlations for each column (size 1000)
    """
    # Ensure input arrays are numpy arrays
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    
    # Subtract the mean of each column (normalize)
    arr1_centered = arr1 - np.mean(arr1, axis=0)
    arr2_centered = arr2 - np.mean(arr2, axis=0)
    
    # Compute the numerator (covariance)
    numerator = np.sum(arr1_centered * arr2_centered, axis=0)
    
    # Compute the denominator (product of standard deviations)
    denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
    
    # Compute the Pearson correlation for each column
    correlation = numerator / denominator
    
    return correlation

# Example usage:
#ARR1 = np.random.rand(370, 1000)  # Replace with your actual matrix
#ARR2 = np.random.rand(370, 1000)  # Replace with your actual matrix
#correlations = correlate_columns(ARR1, ARR2)

In [156]:
indir = '../Data/020-fmriprepped/'
subs_orig = [s for s in os.listdir(indir) if all((s.startswith('sub'),not s.endswith('.html')))]
n_orig = len(subs_orig)
epi_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/func/{sub}_ses-baselineYear1Arm1_task-nback_run-0{r}_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii')
cf_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/anat/analysis_mask_CF.nii')
gm_fn = os.path.join(indir,'{sub}/ses-baselineYear1Arm1/anat/analysis_mask_GM.nii')

subs = [sub for sub in subs_orig if all(( 
    os.path.exists(epi_fn.format(sub=sub,r=1)),
    os.path.exists(epi_fn.format(sub=sub,r=2)),
    os.path.exists(cf_fn.format(sub=sub)),
    os.path.exists(gm_fn.format(sub=sub)),
    os.path.exists(epi_fn.format(sub=sub,r=1).replace('preproc','DEEPCOR'))
    ))]

subs.sort()
print(f'{len(subs)}/{len(subs_orig)}')

34/34


In [8]:
s = 4
r = 1

In [9]:
s = int(s)
r = int(r)

sub = subs[s]
print(sub)

sub-NDARINV6WXU5DEY


In [142]:
brain = ants.image_read(epi_fn.format(sub=sub,r=r).replace('preproc_bold','brain_mask')+'.gz')
#brain = ants.image_read(gm_fn.format(sub=sub))

In [143]:
epi = ants.image_read(epi_fn.format(sub=sub,r=r))
denoised = ants.image_read(epi_fn.format(sub=sub,r=r).replace('preproc','DEEPCOR'))

In [144]:
brain_flat = brain.numpy().flatten()

In [145]:
nt = epi.shape[-1] # Number of timepoints
epi_flat = epi.numpy().reshape(-1,nt).transpose() # (370, 1082035)
denoised_flat = denoised.numpy().reshape(-1,nt).transpose() # (370, 1082035)

In [146]:
corrVals_flat = np.zeros(denoised_flat.shape[1])

In [147]:
nostd1 = epi_flat.std(axis=0)!=0
nostd2 = denoised_flat.std(axis=0)!=0

notflat = nostd1*nostd2

In [148]:
isbrain=brain_flat==1

In [149]:
arr1 = epi_flat[:,isbrain*notflat]
arr2 = denoised_flat[:,isbrain*notflat]

In [150]:
corrVals_flat[isbrain*notflat]=correlate_columns(arr1, arr2)

In [151]:
corrBrain = brain.new_image_like(corrVals_flat.reshape(brain.shape))

In [152]:
corrBrain.to_filename(f'../Data/DeepCorr-inout-corr/{s}-{r}.nii')

In [179]:
for s in tqdm(np.arange(34)):
    for r in [1]:
        s = int(s)
        r = int(r)
        
        sub = subs[s]
        print(sub)
        
        brain = ants.image_read(epi_fn.format(sub=sub,r=r).replace('preproc_bold','brain_mask')+'.gz')
        epi = ants.image_read(epi_fn.format(sub=sub,r=r))
        denoised = ants.image_read(epi_fn.format(sub=sub,r=r).replace('preproc','COMPCORR'))
        brain_flat = brain.numpy().flatten()
        nt = epi.shape[-1] # Number of timepoints
        epi_flat = epi.numpy().reshape(-1,nt).transpose() # (370, 1082035)
        denoised_flat = denoised.numpy().reshape(-1,nt).transpose() # (370, 1082035)
        corrVals_flat = np.zeros(denoised_flat.shape[1])
        nostd1 = epi_flat.std(axis=0)!=0
        nostd2 = denoised_flat.std(axis=0)!=0
        
        notflat = nostd1*nostd2
        isbrain=brain_flat==1
        
        arr1 = epi_flat[:,isbrain*notflat]
        arr2 = denoised_flat[:,isbrain*notflat]
        corrVals_flat[isbrain*notflat]=correlate_columns(arr1, arr2)
        corrBrain = brain.new_image_like(corrVals_flat.reshape(brain.shape))
        corrBrain.to_filename(f'../Data/CompCorr-inout-corr/{s}-{r}.nii')

  0%|                                                    | 0/34 [00:00<?, ?it/s]

sub-NDARINV1H7JEJW1


  3%|█▎                                          | 1/34 [00:19<10:52, 19.79s/it]

sub-NDARINV29P0F670


  6%|██▌                                         | 2/34 [00:39<10:36, 19.88s/it]

...

 94%|████████████████████████████████████████▍  | 32/34 [10:44<00:40, 20.18s/it]

sub-NDARINVXZRDA6CJ


 97%|█████████████████████████████████████████▋ | 33/34 [11:04<00:20, 20.22s/it]

sub-NDARINVZ4YHXYLW


100%|███████████████████████████████████████████| 34/34 [11:24<00:00, 20.14s/it]


In [180]:
ofdir = '../Data/CompCorr-inout-corr/'
files = [f for f in os.listdir(ofdir) if f.endswith('.nii')]
files.sort()

In [181]:
ims = [ants.image_read(os.path.join(ofdir,file)) for file in files]

In [182]:
ims[0].new_image_like(np.array([im.numpy() for im in ims]).mean(axis=0)).to_filename('../Misc/CompCorr-in-out-corr.nii')